In [1]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore")

In [2]:
if not mt5.initialize(path = r'C:\Program Files\Zero Markets MetaTrader 5 Terminal\terminal64.exe'):
    print("initialize() failed")
    mt5.shutdown()


In [26]:
ativo = 'EURUSD'
df1 = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M15, 0,9999))
df1['data'] = pd.to_datetime(df1['time'], unit='s')
df1['data'] = df1['data'].apply(lambda x: str(x)) 
df = df1[df1['data'] > '2022-01-01']
df = df[['data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df

,Data,Open,High,Low,Close,Vol
0,2022-08-10 17:15:00,1.03287,1.03428,1.03242,1.03422,0
1,2022-08-10 17:30:00,1.03422,1.03532,1.03372,1.03492,0
2,2022-08-10 17:45:00,1.03492,1.03563,1.03426,1.03546,0
3,2022-08-10 18:00:00,1.03546,1.03680,1.03513,1.03536,0
4,2022-08-10 18:15:00,1.03536,1.03550,1.03424,1.03502,0
...,...,...,...,...,...,...
9994,2023-01-05 20:00:00,1.05295,1.05327,1.05287,1.05315,0
9995,2023-01-05 20:15:00,1.05316,1.05426,1.05293,1.05365,0
9996,2023-01-05 20:30:00,1.05365,1.05425,1.05352,1.05367,0
9997,2023-01-05 20:45:00,1.05370,1.05390,1.05337,1.05355,0


In [47]:
timeframe = 15
ativo = f'EURUSD{timeframe}'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(rf'C:\Users\Usuário\Documents\Github\Forex-Forecast\Notebooks\Dados\{ativo}.csv', names = colnames, sep='\t')
df = df[df['Data'] > '2022-06-01']

In [48]:
time_period = 20
var_bb = 2
rate_tp = 3
rate_stop = 1
pontos = 0.00010
spread = 0.000005

df['BB_up'],df['BB_mid'],df['BB_low'] = ta.BBANDS(df['Close'], timeperiod=time_period, nbdevup=var_bb, nbdevdn=var_bb, matype=0)

df['lado_candle'] = df['Close'] - df['Open']

df['tamanho_corpo'] = abs(df['Open'] - df['Close'])

df['pos_BB_inicial'] = df.apply(lambda x: 1 if x['Close'] > x['BB_up'] else -1 if x['Close'] < x['BB_low'] else 0, axis = 1)
#df['pos_BB'] = df['pos_BB_inicial'].diff()

#pos_BB = -2 cruzou pra BB_low pra baixo // pos_BB = 2 cruzou BB_high pra cima // 

df['check_inicial'] = df.apply(lambda x: 'call' if (x['pos_BB_inicial'] == -1) else
                                    'sell' if (x['pos_BB_inicial'] == 1) else 0 , axis = 1) 



tempo_lista =10

lista = df['check_inicial'].values
tempo_lista =10
for num, i in enumerate(lista):
    #print(num,i)
    if i != 0:
        for i in range(tempo_lista):
            try:
                n_num = num + i + 1
                lista[n_num] = 0
            except:
                pass

lista_candle = df['lado_candle'].values
lista_acao = [0 for x in range(df.shape[0])]

for num, i in enumerate(lista):
    #print(num,i)
    if i  == 'call':
        n_num = 1
        while True:
            try:
                if lista_candle[num  + n_num] > 0:
                    lista_acao[num  + n_num] = 'call'
                    break
            except:
                pass
            n_num+=1

    elif i  == 'sell':
        n_num = 1
        while True:
            try:
                if lista_candle[num  + n_num] < 0:
                    lista_acao[num  + n_num] = 'sell'
                    break
            except:
                pass
            n_num+=1
df['acao'] = lista_acao

df['stop'] = df.apply(lambda x: x['Close'] + (pontos * rate_stop)  if x['acao'] == 'sell' else 
    x['Close'] - (pontos * rate_stop) if x['acao'] == 'call' else 0, axis = 1)

#rate_tp é somado em 0 unidades de tamanho. rate_tp 2 = tp 2  --Valor igual rate
#rate_tp = 1.5
df['tp'] = df.apply(lambda x: x['Close'] - rate_tp * pontos if x['acao'] == 'sell' else
                                        x['Close'] + rate_tp * pontos if x['acao'] == 'call' else 0, axis =1)


for index, row in df.iterrows():
        if row['acao'] == 'call':
            i = 1
            while True:
                try:
                    if df.loc[index + i,'Low'] <= row['stop']:
                        df.loc[index,'resultado_binario'] = 0 
                        break
                    elif df.loc[index + i,'High'] >= row['tp']:
                        df.loc[index,'resultado_binario'] = 1
                        break
                    i+=1
                except:
                    df.loc[index,'resultado_binario'] = 0
                    break
        elif row['acao'] == 'sell':
            i = 1
            while True:
                try:
                    if df.loc[index + i,'High'] >= row['stop']:
                        df.loc[index,'resultado_binario'] = 0 
                        break
                    elif  df.loc[index + i,'Low'] <= row['tp']:
                        df.loc[index,'resultado_binario'] = 1
                        break
                    i+=1
                except:
                    df.loc[index,'resultado_binario'] = 0
                    break

df_acao = df[(df['acao'] != 0)]

In [49]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')

Timeframe: 15 --pontos: 0.0001 --- rate_tp: 3 -- var_bb: 2
Pontos_liquido: -0.01948 -- Total_operacoes: 456
Agrupado por Hora
   Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0      00             28     0.285714        0.000260
1      01             16     0.375000        0.000720
2      02              6     0.000000       -0.000630
3      03             10     0.300000        0.000150
4      04             25     0.200000       -0.000625
5      05             32     0.156250       -0.001360
6      06             44     0.227273       -0.000620
7      07             24     0.083333       -0.001720
8      08             19     0.157895       -0.000795
9      09             14     0.071429       -0.001070
10     10             16     0.125000       -0.000880
11     11             23     0.043478       -0.002015
12     12             28     0.071429       -0.002140
13     13             25     0.040000       -0.002225
14     14             30     0.166667       -0.001150
15     15 